# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [80]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [81]:
# your code here
orders = pd.read_csv("Orders.csv")

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [82]:
# your code here
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


In [83]:
#aggregate the amount_spent for unique customers
orders_grouped = orders.groupby(['CustomerID'])['amount_spent'].agg(sum)
orders_grouped.head()

CustomerID
12346    77183.60
12347     4310.00
12348     1797.24
12349     1757.55
12350      334.40
Name: amount_spent, dtype: float64

In [84]:
quantiles = orders_grouped.quantile([0.75, 0.95])
print(quantiles)

0.75    1661.640
0.95    5840.182
Name: amount_spent, dtype: float64


In [85]:
orders_grouped = orders.groupby(['CustomerID'])['amount_spent'].agg(sum).reset_index()
orders_grouped.head()

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [86]:
#Create a function to label selected customers as "VIP" or "Preferred"

def costumer(row):
  if row['amount_spent'] >= quantiles.iloc[1]:
    return 'VIP'
  elif (row['amount_spent'] >= quantiles.iloc[0]) & (row['amount_spent'] < quantiles.iloc[1]):
    return 'Preferred'
  else:
    return 'Usual'

orders_grouped['selected_costumers'] = orders_grouped.apply(costumer,axis=1)
orders_grouped.head()

,CustomerID,amount_spent,selected_costumers
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
4,12350,334.40,Usual


In [87]:
# drop Ususal to have a final list of only VIP and Preferred costumers
orders_grouped = orders_grouped[orders_grouped['selected_costumers'].str.contains('Usual')==False ]
orders_grouped.head()

,CustomerID,amount_spent,selected_costumers
0,12346,77183.60,VIP
1,12347,4310.00,Preferred
2,12348,1797.24,Preferred
3,12349,1757.55,Preferred
5,12352,2506.04,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [91]:
display(orders_grouped.shape)

(1085, 3)

In [95]:
costumer_country = orders.groupby(['CustomerID'])['Country'].first().reset_index()
costumer_country.head()

,CustomerID,Country
0,12346,United Kingdom
1,12347,Iceland
2,12348,Finland
3,12349,Italy
4,12350,Norway


In [96]:
display(costumer_country.shape)

(4339, 2)

In [97]:
# Merge the grouped dataframe with orders to have the country

orders_grouped_country = pd.merge(left = orders_grouped,
                                  right = costumer_country,
                                  how = "left", 
                                  left_on = "CustomerID", 
                                  right_on= "CustomerID")
display(orders_grouped_country.shape)
orders_grouped_country.head()

(1085, 4)

,CustomerID,amount_spent,selected_costumers,Country
0,12346,77183.60,VIP,United Kingdom
1,12347,4310.00,Preferred,Iceland
2,12348,1797.24,Preferred,Finland
3,12349,1757.55,Preferred,Italy
4,12352,2506.04,Preferred,Norway


In [107]:
# Filter the dataset to only include VIP customers
VIP = orders_grouped_country[orders_grouped_country['selected_costumers'] == 'VIP']
#print(VIP)

# Group by country and count the number of VIP customers in each country
vip_by_country = VIP.groupby('Country')['selected_costumers'].count()
#print(vip_by_country)

# Sort in descending order based on the count of VIP customers
vip_by_country = vip_by_country.sort_values(ascending=False)
vip_by_country.head()

Country
United Kingdom    177
Germany            10
France              9
Switzerland         3
Spain               2
Name: selected_costumers, dtype: int64

In [108]:
print('The country with the most VIP customers is:', vip_by_country.index[0])

The country with the most VIP customers is: United Kingdom


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [110]:
# I already removed from the data the ones that are not VIP or Preferred
# Filter the dataset to only include VIP customers

# Group by country and count the number of VIP customers in each country
grouped_countries = orders_grouped_country.groupby('Country')['selected_costumers'].count()
#print(vip_by_country)

# Sort in descending order based on the count of VIP customers
grouped_countries = grouped_countries.sort_values(ascending=False)

grouped_countries.head()

Country
United Kingdom    932
Germany            39
France             29
Belgium            11
Switzerland         9
Name: selected_costumers, dtype: int64

In [111]:
print('The country with the most the most VIP+Preferred Customers combined is:', grouped_countries.index[0])

The country with the most the most VIP+Preferred Customers combined is: United Kingdom
